In [4]:
import tkinter as tk
import win32com.client,win32gui
from time import sleep
dm = win32com.client.Dispatch('dm.dmsoft')
window_info = {"title": "Ragnarok", "class_name": ""} # 搜尋 標題 類名
Path ="Astral-Tale"
path = dm.GetBasePath()+Path
dm.SetPath(path)
print('附件路徑存放: ', path)
dm_ret = dm.CreateFolder(path)
if dm_ret :
    print('創建指定目錄: ', dm_ret)
print('插件版本:'+dm.ver())

def find_windows(window_info): # 遍歷 標題 類名
    window_handles = [] # 清空
    def callback(hwnd, param):
        title, class_name = win32gui.GetWindowText(hwnd), win32gui.GetClassName(hwnd)
        if window_info["title"] in title and window_info["class_name"] in class_name:
            param.append(hwnd)
    win32gui.EnumWindows(callback, window_handles)
    return window_handles

hwnd = find_windows(window_info)
print(f"尋找到窗口句柄{hwnd}")

class BindingApp:
    def __init__(self, master):
        self.master = master
        master.title(f"綁定模式測試 窗口: {hwnd[0]}")

        self.display_options = ["gdi", "gdi2", "dx", "dx2", "dx3"]
        self.mouse_options = ["windows", "windows2", "windows3", "dx", "dx2"]
        self.keypad_options = ["windows", "dx"]
        self.mode_options = [0, 2, 4]

        self.display_var = tk.StringVar(master, value=self.display_options[0])
        self.mouse_var = tk.StringVar(master, value=self.mouse_options[0])
        self.keypad_var = tk.StringVar(master, value=self.keypad_options[0])
        self.mode_var = tk.IntVar(master, value=self.mode_options[0])

        self.create_widgets()

    def create_widgets(self):
        self.display_label = tk.Label(self.master, text="圖色:")
        self.display_label.grid(row=0, column=0)
        self.display_menu = tk.OptionMenu(self.master, self.display_var, *self.display_options)
        self.display_menu.grid(row=0, column=1)

        self.mouse_label = tk.Label(self.master, text="滑鼠:")
        self.mouse_label.grid(row=1, column=0)
        self.mouse_menu = tk.OptionMenu(self.master, self.mouse_var, *self.mouse_options)
        self.mouse_menu.grid(row=1, column=1)

        self.keypad_label = tk.Label(self.master, text="鍵盤:")
        self.keypad_label.grid(row=2, column=0)
        self.keypad_menu = tk.OptionMenu(self.master, self.keypad_var, *self.keypad_options)
        self.keypad_menu.grid(row=2, column=1)

        self.mode_label = tk.Label(self.master, text="模式:")
        self.mode_label.grid(row=3, column=0)
        self.mode_menu = tk.OptionMenu(self.master, self.mode_var, *self.mode_options)
        self.mode_menu.grid(row=3, column=1)

        self.result_text = tk.Text(self.master, height=10, width=90)
        self.result_text.grid(row=4, columnspan=2)

        self.start_button = tk.Button(self.master, text="開始測試", command=self.start_test)
        self.start_button.grid(row=5, columnspan=2)

    def start_test(self):
        display = self.display_var.get()
        mouse = self.mouse_var.get()
        keypad = self.keypad_var.get()
        mode = self.mode_var.get()

        dm_ret = dm.BindWindow(hwnd[0], display, mouse, keypad, mode) # 設定綁定
        if dm_ret:
            dm_ret = dm.Capture(0,0,2000,2000,f"'{display}'.bmp") # 截圖
            if dm_ret:
                self.result_text.insert(tk.END, f"截圖保存成功{display}\n")
                self.result_text.update()
            dm.MoveTo(100,100)
            mousexy=dm.GetCursorPos()
            dm.KeyPress(13)
            dm.MoveTo(0,0)
            if mousexy[1] > 0:
                self.result_text.insert(tk.END, f"綁定成功 {mousexy} Display: {display}, Mouse: {mouse}, Keypad: {keypad}, Mode: {mode}\n")
                self.result_text.update()
            dm.UnBindWindow() # 解除綁定
        else:
            self.result_text.insert(tk.END, f"綁定失敗 Display: {display}, Mouse: {mouse}, Keypad: {keypad}, Mode: {mode}\n")
            self.result_text.update()


root = tk.Tk()
app = BindingApp(root)
root.mainloop()

附件路徑存放:  c:\dm\Astral-Tale
創建指定目錄:  1
插件版本:3.1233
尋找到窗口句柄[4789056]
